In [145]:
import pandas as pd
import numpy as np
df=pd.read_csv('tiempos_por_tecla.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232203 entries, 0 to 232202
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   tiempo_id      232203 non-null  int64
 1   usuario        232203 non-null  int64
 2   practica       232203 non-null  int64
 3   tecla          232203 non-null  int64
 4   up             232203 non-null  int64
 5   down           232203 non-null  int64
 6   anterior       232203 non-null  int64
 7   anterior_down  232203 non-null  int64
 8   modo           232203 non-null  int64
dtypes: int64(9)
memory usage: 15.9 MB


In [146]:
DF = df
DF = DF[['usuario','practica','modo','tecla','up','down']]

In [147]:
df['tiempo'] = df.down - df.anterior_down
df['anterior_up'] = df.up.shift(periods = 1)
df['anterior_up'] = np.where(df['down'] == 0, 0, df['anterior_up'])
df['solapa'] = np.where((df.anterior_up - df.down) > 0, 1, 0)
df['tecla'] = np.where(df.tecla == 32, 95, df.tecla)
df['anterior'] = np.where(df.anterior == 32, 95, df.anterior)
df['tecla'] = df['tecla'].apply(chr)
df['anterior'] = df['anterior'].apply(chr)
df['par'] = df.anterior + df.tecla
pt = df.groupby(by=['usuario','practica','modo'])['tiempo'].mean()
pt = pt.reset_index()
pt = pt.rename(columns={'tiempo':'mean'})
pa = df.groupby(by=['usuario','practica','modo'])['solapa'].sum()
pa = pa.reset_index()
pl = df.groupby(by=['usuario','practica','modo','par'])['tiempo'].mean()
pl = pl.reset_index()
pt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   usuario   951 non-null    int64  
 1   practica  951 non-null    int64  
 2   modo      951 non-null    int64  
 3   mean      951 non-null    float64
dtypes: float64(1), int64(3)
memory usage: 29.8 KB


In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232203 entries, 0 to 232202
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tiempo_id      232203 non-null  int64  
 1   usuario        232203 non-null  int64  
 2   practica       232203 non-null  int64  
 3   tecla          232203 non-null  object 
 4   up             232203 non-null  int64  
 5   down           232203 non-null  int64  
 6   anterior       232203 non-null  object 
 7   anterior_down  232203 non-null  int64  
 8   modo           232203 non-null  int64  
 9   tiempo         232203 non-null  int64  
 10  anterior_up    232203 non-null  float64
 11  solapa         232203 non-null  int32  
 12  par            232203 non-null  object 
dtypes: float64(1), int32(1), int64(8), object(3)
memory usage: 22.1+ MB


In [149]:
pl = pl.pivot(index=['usuario','practica','modo'], columns='par', values='tiempo')
pl = pl.reset_index()

In [150]:
ps = df.groupby(by=['usuario','practica','modo'])['tiempo'].std()
ps = ps.reset_index()
ps = ps.rename(columns={'tiempo':'std'})
ps

,usuario,practica,modo,std
0,1,1,1,247.187213
1,1,1,2,232.886798
2,1,1,3,254.744370
3,1,2,1,184.889401
4,1,2,2,200.028626
...,...,...,...,...
946,95,1,2,473.815904
947,96,1,2,518.964295
948,97,1,2,413.758821
949,97,2,2,368.950127


In [151]:
pl

par,usuario,practica,modo, E,",J",",_",.0,.1,.5,.A,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,1,0.000000,NaN,895.5,224.0,320.0,279.5,NaN,...,168.0,290.0,315.666667,211.0,140.0,340.000000,206.0,848.0,184.0,111.5
1,1,1,2,0.000000,1059.0,159.0,255.0,271.0,297.0,1009.0,...,208.0,NaN,158.000000,383.0,179.4,176.500000,226.0,305.0,206.5,167.5
2,1,1,3,194.058333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,1,0.000000,NaN,200.0,353.0,255.0,264.5,NaN,...,168.0,464.0,352.333333,224.0,149.8,196.666667,160.0,430.0,678.5,127.5
4,1,2,2,0.000000,1029.0,145.0,238.0,399.0,294.5,928.0,...,152.0,NaN,145.000000,544.0,128.2,279.500000,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,1,2,0.000000,1648.0,1288.0,328.0,152.0,1064.0,792.0,...,436.0,NaN,152.000000,192.0,204.8,120.000000,160.0,152.0,183.5,144.0
947,96,1,2,0.000000,3575.0,728.0,863.0,1419.0,1308.0,1365.0,...,587.5,NaN,261.000000,584.0,751.6,952.500000,302.0,450.0,258.0,312.5
948,97,1,2,0.000000,1221.0,697.0,542.0,398.0,627.0,1121.0,...,438.0,NaN,260.000000,710.0,524.0,154.500000,1191.0,214.0,387.0,755.0
949,97,2,2,0.000000,1260.0,509.0,1254.0,379.0,421.5,1001.0,...,343.5,NaN,149.000000,1239.0,195.0,163.000000,629.0,294.0,319.0,311.0


In [152]:
pp = df.merge(pt, on=['usuario','practica','modo'], how='outer')
pp = pp.merge(ps, on=['usuario','practica','modo'], how='outer')
pp['lec'] = np.where((pp['mean'] + pp['std']) < pp['tiempo'], 1, 0) 
pp

,tiempo_id,usuario,practica,tecla,up,down,anterior,anterior_down,modo,tiempo,anterior_up,solapa,par,mean,std,lec
0,1,1,1,E,43,0, ,0,1,0,0.0,0, E,208.613546,247.187213,0
1,2,1,1,S,237,157,E,0,1,157,43.0,0,ES,208.613546,247.187213,0
2,3,1,1,T,458,330,S,157,1,173,237.0,0,ST,208.613546,247.187213,0
3,4,1,1,A,571,426,T,330,1,96,458.0,1,TA,208.613546,247.187213,0
4,5,1,1,_,650,572,A,426,1,146,571.0,0,A_,208.613546,247.187213,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232198,232199,79,4,E,43906,43786,I,43735,2,51,43826.0,1,IE,178.169355,204.513255,0
232199,232200,79,4,N,43975,43878,E,43786,2,92,43906.0,1,EN,178.169355,204.513255,0
232200,232201,79,4,_,44157,44049,N,43878,2,171,43975.0,0,N_,178.169355,204.513255,0
232201,232202,79,4,E,44100,44100,_,44049,2,51,44157.0,1,_E,178.169355,204.513255,0


In [153]:
pe = pp.groupby(by=['usuario','practica','modo'])['lec'].sum()
pe = pe.reset_index()
pe

,usuario,practica,modo,lec
0,1,1,1,17
1,1,1,2,16
2,1,1,3,6
3,1,2,1,19
4,1,2,2,18
...,...,...,...,...
946,95,1,2,22
947,96,1,2,30
948,97,1,2,29
949,97,2,2,32


In [154]:
pc = pp.groupby(by=['usuario','practica','modo'])['lec'].count()
pc = pc.reset_index()
pc = pc.rename(columns={'lec': 'cant'})
pc

,usuario,practica,modo,cant
0,1,1,1,251
1,1,1,2,248
2,1,1,3,120
3,1,2,1,251
4,1,2,2,248
...,...,...,...,...
946,95,1,2,248
947,96,1,2,248
948,97,1,2,248
949,97,2,2,248


In [155]:
pe = pe.merge(pc, on=['usuario','practica','modo'])
pe

,usuario,practica,modo,lec,cant
0,1,1,1,17,251
1,1,1,2,16,248
2,1,1,3,6,120
3,1,2,1,19,251
4,1,2,2,18,248
...,...,...,...,...,...
946,95,1,2,22,248
947,96,1,2,30,248
948,97,1,2,29,248
949,97,2,2,32,248


In [156]:
pe['mean_error'] = pe['cant'] / pe['lec']
pe

,usuario,practica,modo,lec,cant,mean_error
0,1,1,1,17,251,14.764706
1,1,1,2,16,248,15.500000
2,1,1,3,6,120,20.000000
3,1,2,1,19,251,13.210526
4,1,2,2,18,248,13.777778
...,...,...,...,...,...,...
946,95,1,2,22,248,11.272727
947,96,1,2,30,248,8.266667
948,97,1,2,29,248,8.551724
949,97,2,2,32,248,7.750000


In [157]:
pt

,usuario,practica,modo,mean
0,1,1,1,208.613546
1,1,1,2,212.379032
2,1,1,3,194.058333
3,1,2,1,191.752988
4,1,2,2,195.750000
...,...,...,...,...
946,95,1,2,274.895161
947,96,1,2,445.762097
948,97,1,2,339.149194
949,97,2,2,306.794355


In [158]:
ps

,usuario,practica,modo,std
0,1,1,1,247.187213
1,1,1,2,232.886798
2,1,1,3,254.744370
3,1,2,1,184.889401
4,1,2,2,200.028626
...,...,...,...,...
946,95,1,2,473.815904
947,96,1,2,518.964295
948,97,1,2,413.758821
949,97,2,2,368.950127


In [159]:
pl

par,usuario,practica,modo, E,",J",",_",.0,.1,.5,.A,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,1,0.000000,NaN,895.5,224.0,320.0,279.5,NaN,...,168.0,290.0,315.666667,211.0,140.0,340.000000,206.0,848.0,184.0,111.5
1,1,1,2,0.000000,1059.0,159.0,255.0,271.0,297.0,1009.0,...,208.0,NaN,158.000000,383.0,179.4,176.500000,226.0,305.0,206.5,167.5
2,1,1,3,194.058333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,1,0.000000,NaN,200.0,353.0,255.0,264.5,NaN,...,168.0,464.0,352.333333,224.0,149.8,196.666667,160.0,430.0,678.5,127.5
4,1,2,2,0.000000,1029.0,145.0,238.0,399.0,294.5,928.0,...,152.0,NaN,145.000000,544.0,128.2,279.500000,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,1,2,0.000000,1648.0,1288.0,328.0,152.0,1064.0,792.0,...,436.0,NaN,152.000000,192.0,204.8,120.000000,160.0,152.0,183.5,144.0
947,96,1,2,0.000000,3575.0,728.0,863.0,1419.0,1308.0,1365.0,...,587.5,NaN,261.000000,584.0,751.6,952.500000,302.0,450.0,258.0,312.5
948,97,1,2,0.000000,1221.0,697.0,542.0,398.0,627.0,1121.0,...,438.0,NaN,260.000000,710.0,524.0,154.500000,1191.0,214.0,387.0,755.0
949,97,2,2,0.000000,1260.0,509.0,1254.0,379.0,421.5,1001.0,...,343.5,NaN,149.000000,1239.0,195.0,163.000000,629.0,294.0,319.0,311.0


In [160]:
final = pe.merge(pt, on=['usuario','practica','modo'])
final = final.merge(pa, on=['usuario','practica','modo'])
final['prom_solapa'] = final['cant'] / final['solapa'] 
final = final.merge(pl, on=['usuario','practica','modo'])

In [161]:
final = final.drop(columns=['lec','cant','solapa','practica'])
final

,usuario,modo,mean_error,mean,prom_solapa, E,",J",",_",.0,.1,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,14.764706,208.613546,4.048387,0.000000,NaN,895.5,224.0,320.0,...,168.0,290.0,315.666667,211.0,140.0,340.000000,206.0,848.0,184.0,111.5
1,1,2,15.500000,212.379032,4.000000,0.000000,1059.0,159.0,255.0,271.0,...,208.0,NaN,158.000000,383.0,179.4,176.500000,226.0,305.0,206.5,167.5
2,1,3,20.000000,194.058333,2.790698,194.058333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,13.210526,191.752988,4.403509,0.000000,NaN,200.0,353.0,255.0,...,168.0,464.0,352.333333,224.0,149.8,196.666667,160.0,430.0,678.5,127.5
4,1,2,13.777778,195.750000,3.542857,0.000000,1029.0,145.0,238.0,399.0,...,152.0,NaN,145.000000,544.0,128.2,279.500000,173.0,336.0,200.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,2,11.272727,274.895161,2.032787,0.000000,1648.0,1288.0,328.0,152.0,...,436.0,NaN,152.000000,192.0,204.8,120.000000,160.0,152.0,183.5,144.0
947,96,2,8.266667,445.762097,19.076923,0.000000,3575.0,728.0,863.0,1419.0,...,587.5,NaN,261.000000,584.0,751.6,952.500000,302.0,450.0,258.0,312.5
948,97,2,8.551724,339.149194,3.594203,0.000000,1221.0,697.0,542.0,398.0,...,438.0,NaN,260.000000,710.0,524.0,154.500000,1191.0,214.0,387.0,755.0
949,97,2,7.750000,306.794355,3.594203,0.000000,1260.0,509.0,1254.0,379.0,...,343.5,NaN,149.000000,1239.0,195.0,163.000000,629.0,294.0,319.0,311.0


In [162]:
prod = final
prod = prod.where( prod >= 0 , prod['mean'], axis=0)
prod = prod.where( prod != np.inf , 0, axis=0)
prod

,usuario,modo,mean_error,mean,prom_solapa, E,",J",",_",.0,.1,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,14.764706,208.613546,4.048387,0.000000,208.613546,895.500000,224.000000,320.000000,...,168.000000,290.000000,315.666667,211.000000,140.000000,340.000000,206.000000,848.000000,184.000000,111.500000
1,1,2,15.500000,212.379032,4.000000,0.000000,1059.000000,159.000000,255.000000,271.000000,...,208.000000,212.379032,158.000000,383.000000,179.400000,176.500000,226.000000,305.000000,206.500000,167.500000
2,1,3,20.000000,194.058333,2.790698,194.058333,194.058333,194.058333,194.058333,194.058333,...,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333
3,1,1,13.210526,191.752988,4.403509,0.000000,191.752988,200.000000,353.000000,255.000000,...,168.000000,464.000000,352.333333,224.000000,149.800000,196.666667,160.000000,430.000000,678.500000,127.500000
4,1,2,13.777778,195.750000,3.542857,0.000000,1029.000000,145.000000,238.000000,399.000000,...,152.000000,195.750000,145.000000,544.000000,128.200000,279.500000,173.000000,336.000000,200.000000,144.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,2,11.272727,274.895161,2.032787,0.000000,1648.000000,1288.000000,328.000000,152.000000,...,436.000000,274.895161,152.000000,192.000000,204.800000,120.000000,160.000000,152.000000,183.500000,144.000000
947,96,2,8.266667,445.762097,19.076923,0.000000,3575.000000,728.000000,863.000000,1419.000000,...,587.500000,445.762097,261.000000,584.000000,751.600000,952.500000,302.000000,450.000000,258.000000,312.500000
948,97,2,8.551724,339.149194,3.594203,0.000000,1221.000000,697.000000,542.000000,398.000000,...,438.000000,339.149194,260.000000,710.000000,524.000000,154.500000,1191.000000,214.000000,387.000000,755.000000
949,97,2,7.750000,306.794355,3.594203,0.000000,1260.000000,509.000000,1254.000000,379.000000,...,343.500000,306.794355,149.000000,1239.000000,195.000000,163.000000,629.000000,294.000000,319.000000,311.000000


In [163]:
#prod['usuario'] = np.where(prod['usuario']==1, True, False)
prod

,usuario,modo,mean_error,mean,prom_solapa, E,",J",",_",.0,.1,...,_I,_J,_L,_O,_P,_Q,_S,_T,_U,_V
0,1,1,14.764706,208.613546,4.048387,0.000000,208.613546,895.500000,224.000000,320.000000,...,168.000000,290.000000,315.666667,211.000000,140.000000,340.000000,206.000000,848.000000,184.000000,111.500000
1,1,2,15.500000,212.379032,4.000000,0.000000,1059.000000,159.000000,255.000000,271.000000,...,208.000000,212.379032,158.000000,383.000000,179.400000,176.500000,226.000000,305.000000,206.500000,167.500000
2,1,3,20.000000,194.058333,2.790698,194.058333,194.058333,194.058333,194.058333,194.058333,...,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333,194.058333
3,1,1,13.210526,191.752988,4.403509,0.000000,191.752988,200.000000,353.000000,255.000000,...,168.000000,464.000000,352.333333,224.000000,149.800000,196.666667,160.000000,430.000000,678.500000,127.500000
4,1,2,13.777778,195.750000,3.542857,0.000000,1029.000000,145.000000,238.000000,399.000000,...,152.000000,195.750000,145.000000,544.000000,128.200000,279.500000,173.000000,336.000000,200.000000,144.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
946,95,2,11.272727,274.895161,2.032787,0.000000,1648.000000,1288.000000,328.000000,152.000000,...,436.000000,274.895161,152.000000,192.000000,204.800000,120.000000,160.000000,152.000000,183.500000,144.000000
947,96,2,8.266667,445.762097,19.076923,0.000000,3575.000000,728.000000,863.000000,1419.000000,...,587.500000,445.762097,261.000000,584.000000,751.600000,952.500000,302.000000,450.000000,258.000000,312.500000
948,97,2,8.551724,339.149194,3.594203,0.000000,1221.000000,697.000000,542.000000,398.000000,...,438.000000,339.149194,260.000000,710.000000,524.000000,154.500000,1191.000000,214.000000,387.000000,755.000000
949,97,2,7.750000,306.794355,3.594203,0.000000,1260.000000,509.000000,1254.000000,379.000000,...,343.500000,306.794355,149.000000,1239.000000,195.000000,163.000000,629.000000,294.000000,319.000000,311.000000


In [164]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model        
from sklearn import neighbors           
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import linear_model
X=prod.drop(axis=1,columns='usuario')  
y=prod['usuario']      
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123)
X_val_train,X_val_test,y_val_train,y_val_test=train_test_split(X_train,y_train,test_size=0.20,random_state=123)

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='uniform')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN
    
print('Knn con peso uniforme')
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))

k_valores=np.arange(1,51,1)   
k_max=0
MRSEkNN_max=0
for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='distance')
    modelokNN.fit(X_val_train, y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    MRSEkNN=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNN>MRSEkNN_max:
        k_max=k
        MRSEkNN_max=MRSEkNN

print('KNn con un peso en distancia')    
print('Valor de k: ', k_max)
print("Mejor MRSE: ", round(MRSEkNN_max,3))



profundidades=np.arange(1,201,1)
p_max=0               
MRSEkNNT_max=0

print("Evaluación modelo Árbol de Decisión en el Validation")

for p in profundidades: 
    modeloT=DecisionTreeClassifier(max_depth=p, random_state=123)        
    modeloT.fit(X_val_train, y_val_train)             
    y_val_pred = modeloT.predict(X_val_test)           
    MRSEkNNT=accuracy_score(y_val_test, y_val_pred)
    
    if  MRSEkNNT>MRSEkNNT_max:
        p_max=p
        MRSEkNNT_max=MRSEkNNT

print('Valor de k: ', p_max)
print("Mejor MRSE: ", round(MRSEkNNT_max,3))




Knn con peso uniforme
Valor de k:  1
Mejor MRSE:  0.605
KNn con un peso en distancia
Valor de k:  1
Mejor MRSE:  0.605
Evaluación modelo Árbol de Decisión en el Validation
Valor de k:  16
Mejor MRSE:  0.842


In [169]:
import random
res = [random.randrange(0, 951, 1) for i in range(20)]

[95,
 640,
 75,
 521,
 0,
 468,
 287,
 85,
 156,
 198,
 763,
 112,
 73,
 898,
 658,
 53,
 922,
 184,
 101,
 156]

In [172]:
for k in res:
    print(prod[prod.index == k])

    usuario  modo  mean_error        mean  prom_solapa    E          ,J  \
95       64     1   13.210526  165.003984     3.217949  0.0  165.003984   

       ,_     .0     .1  ...     _I     _J          _L     _O     _P  \
95  147.0  281.0  255.0  ...  197.0  440.0  159.666667  151.0  151.4   

            _Q    _S     _T     _U     _V  
95  125.333333  96.0  168.0  155.5  108.5  

[1 rows x 145 columns]
     usuario  modo  mean_error        mean  prom_solapa    E      ,J     ,_  \
640       82     2    9.538462  244.100806     2.455446  0.0  1088.0  196.0   

        .0     .1  ...     _I          _J     _L     _O     _P    _Q    _S  \
640  122.0  146.0  ...  779.5  244.100806  148.0  264.0  194.2  92.0  97.0   

        _T     _U     _V  
640  215.0  186.5  209.0  

[1 rows x 145 columns]
    usuario  modo  mean_error        mean  prom_solapa    E          ,J    ,_  \
75       62     1    8.964286  244.207171     3.585714  0.0  244.207171  78.5   

       .0      .1  ...     _I     _